# modelo 06 - random forest

In [25]:
import pandas as pd

In [26]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [27]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

{'S-unknown', 'openspace', 'r_openspace', 'roof'}

In [28]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

{'D', 'S'}

In [29]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [30]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
127,IDHpqVXgNeP7P0F,[],[],"[1, 1]","[[(269.952, 196.096), (332.16, 236.37333333333...","[1, 0, 0, 0, 1, 0]"
996,IDZJJxMc9j,[],[],"[1, 1, 1]","[[(439.624, 417.736), (427.336, 388.04), (451....","[1, 0, 0, 0, 1, 0]"
457,ID0w0xWq7fmB,[],[],[1],"[[(263.68, 266.92266666666666), (258.048, 222....","[1, 0, 0, 0, 1, 0]"


---

In [31]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [32]:
# Load EfficientNetB0 without the top layer
effnet = EfficientNetB5(weights="imagenet", include_top=False, pooling="avg")

datagen = ImageDataGenerator(
    rotation_range=40,       # Random rotation range in degrees
    width_shift_range=0.2,   # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,         # Random shear transformation
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest',      # How to fill in newly created pixels (nearest pixel)
    brightness_range=[0.2, 1.8],
    
)

# Function to extract features from an image
def extract_image_features(image_path):

    image_path = "images/" + image_path + ".jpg"

    img = cv2.imread(image_path)
    img = cv2.resize(img, (456, 456))
    img = preprocess_input(img)  # Normalize image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    augmented_img = datagen.random_transform(img[0].copy())
    augmented_img = np.expand_dims(augmented_img, axis=0)  # Add batch dimension


    features = effnet.predict(augmented_img, verbose=0)  # Extract features
    return features.flatten()  # Convert to 1D

In [33]:
"""image_features = []
additional_features = []
targets = []
for i, row in df_train.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(df_train)}")

    image_features.append(extract_image_features(row["img_id"]))
    additional_features.append(row["more_features"])

    nr_boil = sum(row["nr_boil"])
    nr_pan = sum(row["nr_pan"])
    targets.append([nr_boil, nr_pan])

additional_features = np.array(additional_features)
targets = np.array(targets)

X_train = np.hstack((image_features, additional_features))
y_train = targets"""
''

''

In [34]:
"""image_features = []
additional_features = []
targets = []
for i, row in df_val.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(df_val)}")

    image_features.append(extract_image_features(row["img_id"]))
    additional_features.append(row["more_features"])

    nr_boil = sum(row["nr_boil"])
    nr_pan = sum(row["nr_pan"])
    targets.append([nr_boil, nr_pan])

additional_features = np.array(additional_features)
targets = np.array(targets)

X_val = np.hstack((image_features, additional_features))
y_val = targets"""
''

''

In [35]:
import pickle

"""a = {"X_train": X_train, "y_train": y_train, "X_val": X_val, "y_val": y_val}
with open('ModelBuild/model06_data.pkl', 'wb') as f:
    pickle.dump(a, f)"""

with open('ModelBuild/model06_data.pkl', 'rb') as f:
    a = pickle.load(f)

X_train = a["X_train"]
y_train = a["y_train"]
X_val = a["X_val"]
y_val = a["y_val"]

In [36]:
# Define the parameter grid for RandomizedSearchCV
param_dist = {
    "n_estimators": randint(250, 2500),  # Number of trees
    "max_depth": randint(5, 100),  # Maximum depth of each tree
    "min_samples_split": randint(7, 20),  # Minimum samples to split a node
    "min_samples_leaf": randint(5, 10),  # Minimum samples at leaf node
    "max_features": ["sqrt", "log2", 0.1],  # Number of features to consider for best split
    "bootstrap": [True],
}

param_dist = {'bootstrap': True,
    'max_depth': 34,
    'max_features': 0.1,
    'min_samples_leaf': 7,
    'min_samples_split': 17,
    'n_estimators': 864}

param_dist = {'bootstrap': [False],
    'max_depth': [30],
    'max_features': [0.1],
    'min_samples_leaf': [7],
    'min_samples_split': [20],
    'n_estimators': [1000]}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Use RandomizedSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(
    rf, 
    param_distributions=param_dist,
    n_iter=1,
    cv=7,
    scoring="neg_mean_absolute_error", 
    n_jobs=7,
    verbose=2
)

# Fit the model
random_search.fit(X_train, y_train)

Fitting 7 folds for each of 1 candidates, totalling 7 fits
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=18.8min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=18.9min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=19.0min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=19.3min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=19.4min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=1000; total time=19.6min
[CV] END bootstrap=False, max_depth=30, max_features=0.1, min_samples_leaf=7, min_samples_split=20, n_estimators=

RandomizedSearchCV(cv=7, estimator=RandomForestRegressor(random_state=42),
                   n_iter=1, n_jobs=7,
                   param_distributions={'bootstrap': [False], 'max_depth': [30],
                                        'max_features': [0.1],
                                        'min_samples_leaf': [7],
                                        'min_samples_split': [20],
                                        'n_estimators': [1000]},
                   scoring='neg_mean_absolute_error', verbose=2)

In [37]:
#results_df = pd.DataFrame(random_search.cv_results_)
#results_df

In [38]:
best_params = random_search.best_params_
best_params

{'n_estimators': 1000,
 'min_samples_split': 20,
 'min_samples_leaf': 7,
 'max_features': 0.1,
 'max_depth': 30,
 'bootstrap': False}

prunning

In [71]:
# Train final model with best parameters
best_rf = random_search.best_estimator_

In [66]:
y_pred = best_rf.predict(X_val)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_val, y_pred)
print(f"Test MAE: {mae}")

y_pred = best_rf.predict(X_train)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_train, y_pred)
print(f"Train MAE: {mae}")

Test MAE: 2.451639344262295
Train MAE: 1.3511685116851169


In [67]:
len(best_rf.estimators_)

91

In [51]:
import numpy as np

# Get feature importances per tree
tree_importances = np.array([tree.feature_importances_.sum() for tree in best_rf.estimators_])

# Define threshold and remove weak trees
threshold = np.percentile(tree_importances, 18)  # Remove lowest 20%
best_rf.estimators_ = [tree for i, tree in enumerate(best_rf.estimators_) if tree_importances[i] > threshold]

In [52]:
len(best_rf.estimators_)

91

In [72]:
y_pred = best_rf.predict(X_val)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_val, y_pred)
print(f"Test MAE: {mae}")

y_pred = best_rf.predict(X_train)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_train, y_pred)
print(f"Train MAE: {mae}")

Test MAE: 2.451639344262295
Train MAE: 1.3511685116851169


---

In [73]:
submission = pd.read_csv("zindi_files/SampleSubmission.csv")
test = pd.read_csv("Test.csv")

In [74]:
test.head()

,ID,img_origin,placement
0,ID00qprY,D,roof
1,ID01AciUc,D,roof
2,ID0328D,D,roof
3,ID05WxObCFTs9,D,roof
4,ID06AdCmLMlkO,S,S-unknown


In [75]:
submission.head()

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,0
2,ID01AciUc_boil,0
3,ID01AciUc_pan,0
4,ID0328D_boil,0


In [76]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["placement"], inplace=True)

hot_encod_img_placement(test)

In [77]:
test.head()

,ID,more_features
0,ID00qprY,"[1, 0, 0, 0, 1, 0]"
1,ID01AciUc,"[1, 0, 0, 0, 1, 0]"
2,ID0328D,"[1, 0, 0, 0, 1, 0]"
3,ID05WxObCFTs9,"[1, 0, 0, 0, 1, 0]"
4,ID06AdCmLMlkO,"[0, 0, 0, 1, 0, 1]"


In [78]:
image_features = []
additional_features = []
targets = []
for i, row in test.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(test)}")

    image_features.append(extract_image_features(row["ID"]))
    additional_features.append(row["more_features"])


additional_features = np.array(additional_features)
targets = np.array(targets)

X_test = np.hstack((image_features, additional_features))

0 of 1107
250 of 1107
500 of 1107
750 of 1107
1000 of 1107


In [79]:
pred = best_rf.predict(X_test)

pred

array([[7.48986129e-02, 2.04960019e+00],
       [1.14593985e-02, 2.78406387e+00],
       [2.01898106e-01, 7.09058690e+00],
       ...,
       [6.81704443e-01, 1.87968211e+01],
       [9.65957563e-02, 4.31549355e+00],
       [1.89324569e-01, 6.50257086e+00]])

In [80]:
for i, row in test.iterrows():
    submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]
    submission.loc[submission["ID"] == f"{row['ID']}_pan", "Target"] = pred[i][1]

submission.head()

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_3200/1570848020.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.07489861292815367' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]


,ID,Target
0,ID00qprY_boil,0.074899
1,ID00qprY_pan,2.049600
2,ID01AciUc_boil,0.011459
3,ID01AciUc_pan,2.784064
4,ID0328D_boil,0.201898


In [81]:
submission.to_csv("zindi_files/model06_submission.csv", index=False)

**public score:** 3.26